In [1]:
import json
import glob
import os
import numpy

In [4]:
def read_json_files(path_to_file):
    with open(path_to_file) as p:
        return json.load(p)
        

def giant_list(json_files):
    return [read_json_files(f) for f in json_files]

def parse_json(json_file):
    if len(json_file['people']) > 0:
        return json_file['people'][0].get('pose_keypoints_2d', [])
    
    return []

def format_XY(XY):
    X = [xy[::3] for xy in XY]
    Y = [xy[1::3] for xy in XY]
    
    return numpy.array(X), numpy.array(Y)

def normalize_input_data(input_data):
    #print(input_data)
    max_d, min_d = numpy.max(input_data), numpy.min(input_data)
    #print(max_d, min_d)
    return (input_data-min_d) / (max_d-min_d)

def preprocessing(file_path, cut_off=75):
    origin_json = giant_list(sorted(glob.glob(file_path), key=os.path.getmtime))
    XY = [parse_json(j) for j in origin_json]
    XY = [xy for xy in XY if xy][:75]
    
    X, Y = format_XY(XY)
    print(X.shape, Y.shape)
    
    X = normalize_input_data(X)
    Y = normalize_input_data(Y)
    return [X, Y]
    
    
def preprocessiongDir(dir_path):
    path = os.listdir(dir_path)
    samples = [preprocessing(dir_path+f) for f in path]
    return numpy.array(samples).reshape[-1]
    
    

In [5]:
file_path1 = "../openpose/output/MyPlayground-52-of-67/*.json"
X1, Y1 = preprocessing(file_path1)
file_path2 = "../openpose/output/MyPlayground-59-of-67/*.json"
X2, Y2 = preprocessing(file_path2)


(75, 25) (75, 25)
(75, 25) (75, 25)


In [6]:
file_path3 = "../openpose/output/MyPlayground-58-of-67/*.json"
X3, Y3 = preprocessing(file_path3)

(75, 25) (75, 25)


In [7]:
input_1 = numpy.array([X1, Y1]).reshape(-1)
input_2 = numpy.array([X2, Y2]).reshape(-1)

In [8]:
train_data = numpy.array([input_1, input_2])

In [9]:
train_data.shape

(2, 3750)

In [10]:
train_target = numpy.array([1,1])

In [11]:
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

#MLP model structre
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=3750))
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_target, 
          epochs=30,batch_size=16,verbose=1)

score = model.evaluate(tensor_test, test_target)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               960256    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_3 (Dense)              (None, 16)                2064      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 17        
Total params: 995,233
Trainable params: 995,233
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/30
2/2 [==============================] - 0s 128ms/step - loss: 0.6756 - acc: 0.5000
Epoch 2/30
2/2 [==============================] - 0s 3m

NameError: name 'tensor_test' is not defined